In [1]:
import json
import pandas as pd
import boto3
from datetime import datetime, timedelta

In [2]:
def load_youtube_history(json_path: str) -> list:
    """
    Load the YouTube watch history JSON file
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [ ]:
def extract_relevant_fields(data: list) -> pd.DataFrame:
    """
    Extract channel name and time from the raw JSON data
    """
    records = []
    
    # Loop through every entry within the JSON file
    for entry in data:
        if 'subtitles' in entry:
            channel_name = entry['subtitles'][0].get('name')
        else:
            channel_name = None
        
        time_str = entry.get('time')
        # Appnd channel name and time
        records.append({
            'channel_name': channel_name,
            'time': time_str
        })
    
    return pd.DataFrame(records)

In [ ]:
def convert_to_local_time(utc_time_str: str) -> datetime:
    """
    Convert UTC timestamp (from Google Takeout) to local time:
    - Before August 28, 2025: UK time (UTC+1)
    - After August 28, 2025: Malaysia time (UTC+8)
    """
    try:
        utc_time = datetime.strptime(utc_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    # To catch any dates that have misaligned format
    except ValueError:
        return None 

    if utc_time < datetime(2025, 8, 28):
        # UK local time
        return utc_time + timedelta(hours=1)
    else:
        # Malaysian local time
        return utc_time + timedelta(hours=8)

In [11]:
def process_time_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert timestamps into separate date and time columns (local time)
    """

    df = df.copy()

    # Convert all UTC times → local time
    df['local_time'] = df['time'].apply(convert_to_local_time)

    # Drop rows with invalid or missing times
    df = df.dropna(subset=['local_time'])

    # Extract date and time strings
    df['date'] = df['local_time'].dt.strftime('%Y-%m-%d')
    df['time_only'] = df['local_time'].dt.strftime('%H:%M')

    # Keep only relevant columns
    return df[['channel_name', 'date', 'time_only']]

In [6]:
def save_to_csv(df: pd.DataFrame, output_path: str) -> None:
    """
    Save cleaned DataFrame to a CSV file
    """
    df.to_csv(output_path, index=False)
    print(f"Saved cleaned data to: {output_path}")

In [7]:
def upload_to_s3(local_path, bucket, s3_path) -> None:
    """
    Upload preprocessed CSV file to S3 bucket
    """
    s3 = boto3.client('s3')
    try:
        s3.upload_file(local_path, bucket, s3_path)
        print(f"Uploaded {local_path} to s3://{bucket}/{s3_path}")
    except Exception as e:
        print(f"Upload failed: {e}")

In [8]:
raw_data = load_youtube_history("watch-history.json")

In [9]:
df = extract_relevant_fields(raw_data)

In [12]:
df_clean = process_time_columns(df)

In [16]:
# An example of how the structure of the data look like 
df_clean.head(5)

,channel_name,date,time_only
0,Alex The Analyst,2025-10-21,16:18
1,VIEWS FROM SIX8™,2025-10-21,15:56
2,High Peak Autos,2025-10-21,15:49
3,High Peak Autos,2025-10-21,15:42
4,zWORMz Gaming,2025-10-21,15:41


In [13]:
save_to_csv(df_clean, "preprocessed_youtube_watch_history.csv")

Saved cleaned data to: preprocessed_youtube_watch_history.csv


In [14]:
upload_to_s3("preprocessed_youtube_watch_history.csv",
             "aws-projects-bucket-kjs",
             "youtube-watch-history/preprocessed-data/preprocessed_youtube_watch_history.csv")

Uploaded preprocessed_youtube_watch_history.csv to s3://aws-projects-bucket-kjs/youtube-watch-history/preprocessed-data/preprocessed_youtube_watch_history.csv
